## Setup

In [1]:
%load_ext sql

In [2]:
from dotenv import find_dotenv
from dotenv import load_dotenv
load_dotenv(find_dotenv())

True

In [3]:
import google.auth
credentials, project_id = google.auth.default()

In [23]:
import os
from google.cloud.sql.connector import Connector
import sqlalchemy

# initialize Connector object
connector = Connector()

# function to return the database connection object
def get_conn():
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pg8000",
        user=DB_USER,
        password=DB_PASSWORD,
        db=DB_NAME
    )
    return conn

def get_pool():
    # create connection pool with 'creator' argument to our connection object function
    pool = sqlalchemy.create_engine(
        "postgresql+pg8000://",
        creator=get_conn,
    )
    
    return pool

## Create Business Connection

In [24]:
DB_USER = os.getenv("BUSINESS_USERNAME", "")
DB_PASSWORD = os.getenv("BUSINESS_PASSWORD", "")
DB_NAME = os.getenv("BUSINESS_NAME", "")
INSTANCE_CONNECTION_NAME = os.getenv("BUSINESS_CONNECTION", "")

business_pool = get_pool()
%sql business_pool --alias business
%sql business

In [25]:
%%sql
select now();

Running query in 'business'

1 rows affected.

now
2024-04-29 03:20:34.387956+00:00


## Create Warehouse Connection

In [26]:
DB_USER = os.getenv("WAREHOUSE_USERNAME", "")
DB_PASSWORD = os.getenv("WAREHOUSE_PASSWORD", "")
DB_NAME = os.getenv("WAREHOUSE_NAME", "")
INSTANCE_CONNECTION_NAME = os.getenv("WAREHOUSE_CONNECTION", "")

warehouse_pool = get_pool()
%sql warehouse_pool --alias warehouse
%sql warehouse

In [27]:
%%sql
select now();

Running query in 'warehouse'

1 rows affected.

now
2024-04-29 03:22:30.802538+00:00


## List Database Connections

In [34]:
%sql --connections

current,url,alias
*,postgresql+pg8000://,business
,postgresql+pg8000://,warehouse


## Query Business Database

In [35]:
%sql business

In [36]:
%%sql business_data <<
select * from business

Running query in 'business'

2 rows affected.

In [37]:
business_data

id,legal_name,legal_type,identifier,tax_id,nano_id
1,C3 HUB CCC LTD. - IMPORT_TEST,CC,BC1120292,None,abcd123
2,FB HEALTH SUPPORT CCC CORP. - IMPORT_TEST,CC,BC1335427,None,efgh456


# Rollback the New Changes

In [ ]:
identifier = "BC0814603"
delete_stmt = (
  sqlalchemy.delete('filing')
  .where('identifier' == identifier)
)

select_stmt = (
  sqlalchemy.select('filing')
  .where('identifier' == identifier)
)

##Get filing before deleting it
result = business_pool.execute(select_stmt).first()
filingId = result[0].id

#delete filing
del_results = business_pool.execute(delete_stmt)

#Get colin_event_ids
select_stmt = (
  sqlalchemy.select('colin_event_ids')
  .where('filing_id' == filingId)
)
colin_event_ids = business_pool.execute(select_stmt).all()

unique_event_ids = []
for item in colin_event_ids:
    if item["colin_event_id"] not in unique_event_ids:
        unique_event_ids.append(item["colin_event_id"])

#delete colin_event_ids
delete_stmt = (
  sqlalchemy.delete('colin_event_ids')
  .where('filing_id' == filingId)
)

del_results2 = business_pool.execute(delete_stmt)

del_results
del_results2

# Rollback Colin

In [ ]:
import requests
from datetime import datetime

COLIN_API_URL = os.getenv("COLIN_API_URL", "")
COLIN_CLIENT = os.getenv("COLIN_API_SVC_CLIENT_ID", "")
COLIN_SECRET = os.getenv("COLIN_AI_API_SVC_CLIENT_SECRET", "")

colin_api_endpoint = f"{COLIN_API_URL}/reset/by_event_id"

#Get Token
token_url = os.getenv("AUTH_SVC_URL", "")
timeout = int(os.getenv("AUTH_SVC_TIMEOUT", 20))

data = "grant_type=client_credentials"

# get service account token
res = requests.post(
  url=token_url,
  data=data,
  headers={"content-type": "application/x-www-form-urlencoded"},
  auth=(COLIN_CLIENT, COLIN_SECRET),
  timeout=timeout,
)

token = res.json().get("access_token")

colin_business_identifier = (
  identifier[2:] if identifier.startswith("BC", 0, 2) else identifier
)
date = datetime.now().strftime("%Y-%m-%d")

#ADD event ids
reset_data = {
  "event_ids": unique_event_ids,
}


reset_details = requests.post(
    url=colin_api_endpoint,
    auth=("Bearer", token),
    headers={"content-type": "application/json"},
    json={"data": reset_data},
).json()

reset_details

## SBCAUTH Keycloak

This is going to have to be manual since sbcauth only does soft delete and keycloak only supports modification via an admin api key. (Previously this had to be admin on master which won't happen TBD if thats still the case)


Manual - Delete the user in keycloak.

### Find the user id using last name (example based on integration test user)
```sql
select id into user_id_value from users where last_name = 'WU';
```

### Delete from SBC Auth DB
Note replace 2223 below with the user id
```sql
begin;
update entities set modified_by_id = null where modified_by_id = 2223;
update memberships set created_by_id = null where created_by_id = 2223;
delete from product_subscriptions where created_by_id = 2223;
delete from memberships where user_id = 2223;
delete from contact_links where created_by_id = 2223 or user_id = 2223;
delete from affiliations where created_by_id = 2223;
delete from orgs where created_by_id = 2223;
delete from contact_links where user_id = 2223;
delete from contact_links where created_by_id = 2223;
delete from contacts where created_by_id = 2223;
delete from users where id = 2223;
```

